Retrieve experiment results

In [4]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [40]
basedirs = [join(get_output_dir(), 'predict_multi', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        alpha = config['alpha']
        weights = config['dataset_weights']
        beta = config['beta']
        seed = config['seed']
        model = config['model']
        source = config['source']
        score = info['score']
        rank = info['rank']
        normalized = config['with_std']
        res = {'datasets': datasets, 'alpha': alpha, 'beta': beta, 'seed': seed, 'rank': rank,
               'source': source, 'normalized': normalized,
              'model': model}
        for dataset, weight in weights.items():
            res['%s_weight' % dataset] = weight
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

Compare results with trace norm, with and without transfer

In [17]:
df_agg = res.groupby(by=['normalized', 'source', 'model', 'datasets', 'alpha', 'beta', 'archi_weight']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)

transfer_df = df_agg[[('test_archi', 'mean'), ('test_hcp', 'mean'), ('train_hcp', 'mean'), ('train_archi', 'mean')]]
# transfer_df.groupby(level=['normalized', 'source', 'model', 'datasets', 'beta', 'archi_weight']).aggregate('max')

Compare results with 3 datasets

In [18]:
transfer_df

test_archi  \
                                                                                    mean   
normalized source                 model datasets   alpha    beta archi_weight              
False      hcp_rs_positive_single trace archi__hcp 0.000001 0    0.0            0.036752   
                                                                 0.5            0.862393   
                                                                 1.0            0.864103   
                                                                 1.5            0.876923   
                                                                 2.0            0.853846   
                                                   0.000003 0    0.0            0.024786   
                                                                 0.5            0.863248   
                                                                 1.0            0.864957   
                                                                 1.5            0.876068   
                                                                 2.0            0.852991   
                                                   0.000010 0    0.0            0.026496   
                                                                 0.5            0.864957   
                                                                 1.0            0.868376   
                                                                 1.5            0.876923   
                                                                 2.0            0.853846   
                                                   0.000032 0    0.0            0.023932   
                                                                 0.5            0.869231   
                                                                 1.0            0.872650   
                                                                 1.5            0.876068   
                                                                 2.0            0.859829   
                                                   0.000100 0    0.0            0.044444   
                                                                 0.5            0.879487   
                                                                 1.0            0.873504   
                                                                 1.5            0.876923   
                                                                 2.0            0.864103   
                                                   0.000316 0    0.0            0.043590   
                                                                 0.5            0.875214   
                                                                 1.0            0.870085   
                                                                 1.5            0.878632   
                                                                 2.0            0.864957   
                                                   0.001000 0    0.0            0.029060   
                                                                 0.5            0.860684   
                                                                 1.0            0.865812   
                                                                 1.5            0.876923   
                                                                 2.0            0.863248   
                                                   0.003162 0    0.0            0.013675   
                                                                 0.5            0.786325   
                                                                 1.0            0.827350   
                                                                 1.5            0.840171   
                                                                 2.0            0.653846   
                                                   0.010000 0    0.0            0.042735   
                              

In [13]:
this_df

hcp_weight  \
                                                                                    mean   
normalized source                 model datasets   alpha    beta archi_weight              
False      hcp_rs_positive_single trace archi__hcp 0.000001 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000003 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000010 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000032 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000100 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000316 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.001000 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.003162 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.010000 0    0.0                 2.0   
                              

In [5]:
indices.values

array([(False, 'hcp_rs_positive_single', 'logistic', 'archi', 0.0, 0.0001),
       (False, 'hcp_rs_positive_single', 'trace', 'archi', 0.001, 0.0),
       (False, 'hcp_rs_positive_single', 'trace', 'archi__hcp', 0.00031622776601683794, 0.0)], dtype=object)

In [6]:
this_df = df_agg.query("datasets == 'brainomics__archi__hcp'")
test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in ['brainomics', 'archi', 'hcp']) / 3
this_df = this_df.assign(test_mean=test_mean)
indices = this_df[('test_mean', 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
this_df = this_df.loc[indices.values]
dfs['archi+brainomics+hcp'] = this_df

In [7]:
result_df = pd.concat(dfs.values(), keys=dfs.keys(), names=['target'])

In [8]:
result_df

rank  \
                                                                                                         mean   
target               normalized source                 model    datasets               alpha    beta            
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  17.05   
                                                       trace    brainomics             0.003162 0.0000   9.50   
                                                                brainomics__hcp        0.003162 0.0000  19.05   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  29.05   
                                                       trace    archi                  0.001000 0.0000  18.85   
                                                                archi__hcp             0.000316 0.0000  38.05   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  51.45   

                                                                                                                  \
                                                                                                             std   
target               normalized source                 model    datasets               alpha    beta               
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  0.223607   
                                                       trace    brainomics             0.003162 0.0000  0.512989   
                                                                brainomics__hcp        0.003162 0.0000  0.825578   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  0.223607   
                                                       trace    archi                  0.001000 0.0000  0.670820   
                                                                archi__hcp             0.000316 0.0000  1.050063   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  0.944513   

                                                                                                              seed  \
                                                                                                              mean   
target               normalized source                 model    datasets               alpha    beta                 
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  2330081002   
                                                       trace    brainomics             0.003162 0.0000  2330081002   
                                                                brainomics__hcp        0.003162 0.0000  2330081002   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  2330081002   
                                                       trace    archi                  0.001000 0.0000  2330081002   
                                                                archi__hcp             0.000316 0.0000  2330081002   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  2330081002   

                                                                                                                      \
                                                                                                                 std   
target               normalized source                 model    datasets               alpha    beta                   
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  1.242420e+09   
                                                       trace    brainomics             0.003162 0.0000  1.242420e+09   
                                

In [21]:
df_agg.query("datasets == 'brainomics__archi__hcp'")

rank  \
                                                                                   mean   
normalized source                 model datasets               alpha    beta              
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   71.000000   
                                                               0.000001 0.0   71.000000   
                                                               0.000003 0.0   71.000000   
                                                               0.000010 0.0   71.000000   
                                                               0.000032 0.0   71.000000   
                                                               0.000100 0.0   70.666667   
                                                               0.000316 0.0   52.000000   
                                                               0.001000 0.0   28.000000   
                                                               0.003162 0.0   18.333333   
                                                               0.010000 0.0   10.000000   
                                                               0.031623 0.0    3.666667   
                                                               0.100000 0.0    1.000000   
                                                               0.316228 0.0    0.000000   
                                                               1.000000 0.0    0.000000   

                                                                                        \
                                                                                   std   
normalized source                 model datasets               alpha    beta             
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   0.000000   
                                                               0.000001 0.0   0.000000   
                                                               0.000003 0.0   0.000000   
                                                               0.000010 0.0   0.000000   
                                                               0.000032 0.0   0.000000   
                                                               0.000100 0.0   0.577350   
                                                               0.000316 0.0   1.000000   
                                                               0.001000 0.0   0.000000   
                                                               0.003162 0.0   1.154701   
                                                               0.010000 0.0   0.000000   
                                                               0.031623 0.0   0.577350   
                                                               0.100000 0.0   0.000000   
                                                               0.316228 0.0   0.000000   
                                                               1.000000 0.0   0.000000   

                                                                                    seed  \
                                                                                    mean   
normalized source                 model datasets               alpha    beta               
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   1243615929   
                                                               0.000001 0.0   1243615929   
                                                               0.000003 0.0   1243615929   
                                                               0.000010 0.0   1243615929   
                                                               0.000032 0.0   1243615929   
                                                               0.000100 0.0   1243615929   
                                                               0.000316 0.0   1243615929   
                                                               0.001000 0.0   124361

In [22]:
df

NameError: name 'df' is not defined